In [350]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [351]:
df = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
df.info()
df = df.rename(columns = {'anatom_site_general_challenge':'site'})
df.drop(["target"],axis = 1,inplace = True)
df1 = df[:]
print(df)


In [352]:
df.isnull().sum()


In [353]:

df["age_approx"].fillna(value = df["age_approx"].mean(),inplace = True)
df["sex"].fillna(value = df["sex"].mode()[0],inplace = True)
df["site"].fillna(value = df["site"].mode()[0],inplace = True)
df.isnull().sum()


****Data Visualization and Analysis



In [354]:


sex_values = []
for k in range(len(df["benign_malignant"])):
    if df["benign_malignant"][k]=="malignant":
        sex_values.append(df["sex"][k])
se = ["male",'female']
val = [sex_values.count("male"),sex_values.count("female")]
plt.bar(se,val)
plt.xlabel("sex")
plt.ylabel("Count")
plt.show()


#Male suffers more than female

In [355]:
ages = []
for k in range(len(df["benign_malignant"])):
    if df["benign_malignant"][k]=="malignant":
        ages.append(df["age_approx"][k])
sns.displot(ages)

#People age between 50 to 80 suffers more

In [356]:
sites = []
for k in range(len(df["benign_malignant"])):
    if df["benign_malignant"][k]=="malignant":
        sites.append(df["site"][k])
sns.countplot(sites)

# torso is the major site

In [357]:

# Patient id is not useful
df.drop(["patient_id"],axis =1,inplace = True)

ENCODINGS OF  CATEGORICAL FIELDS


In [358]:

df["sex"].replace(["male","female"],[0,1],inplace = True)
df["benign_malignant"].replace(["benign","malignant"],[0,1],inplace = True)
df = pd.get_dummies(df,columns  = ["site","diagnosis"],drop_first = True)
print(df)

CORELATIONS BETWEEN COLUMNS AND BENIGN_MALIGNANT COLUMN

In [359]:
sns.heatmap(df.corr())
print(df.corr())

In [360]:
sns.heatmap(df.corr()[["benign_malignant"]].sort_values('benign_malignant').tail(16),annot = True)

# The benign_malignant column depends on sex,site_upper extremity columns,age_approx

**TRAINING BASED ON IMAGES**

In [361]:
train = []
for k in range(23180):
    train.append([df["diagnosis_melanoma"][k],df["site_oral/genital"][k],df["site_upper extremity"][k],(df["age_approx"][k])])


In [362]:
test = []

for k in range(23180,33126):

    test.append([df["diagnosis_melanoma"][k],df["site_oral/genital"][k],df["site_upper extremity"][k],(df["age_approx"][k])])
    

In [363]:
from sklearn.linear_model import LogisticRegression
X = train
y = df["benign_malignant"][:23180]
classifier_lr = LogisticRegression(solver = 'liblinear')
classifier_lr.fit(X,y)
X_test = test
y_test = df["benign_malignant"][23180:]
y_pred_lr = classifier_lr.predict(X_test)
print(accuracy_score(y_test,y_pred_lr))
print(confusion_matrix(y_test,y_pred_lr))

In [365]:
from sklearn import svm
X = train
y = df["benign_malignant"][:23180]
classifier_svm = svm.SVC()
classifier_svm.fit(X,y)
X_test = test
y_test = df["benign_malignant"][23180:]
y_pred_svm = classifier_svm.predict(X_test)
print(accuracy_score(y_test,y_pred_svm))
print(confusion_matrix(y_test,y_pred_svm))

In [366]:
from sklearn.tree import DecisionTreeClassifier
X = train
y = df["benign_malignant"][:23180]
classifier_dt = DecisionTreeClassifier()
classifier_dt.fit(X,y)
X_test = test
y_test = df["benign_malignant"][23180:]

y_pred_dt = classifier_dt.predict(X_test)
print(accuracy_score(y_test,y_pred_dt))
print(confusion_matrix(y_test,y_pred_dt))

In [367]:
from sklearn.ensemble import RandomForestClassifier
X = train
y = df["benign_malignant"][:23180]
classifier_rf = RandomForestClassifier()
classifier_rf.fit(X,y)
X_test = test
y_test = df["benign_malignant"][23180:]
y_pred_dt = classifier_rf.predict(X_test)
print(accuracy_score(y_test,y_pred_dt))
print(confusion_matrix(y_test,y_pred_dt))


**TESTING USING SUPPORT VECTOR MACHNINE**

In [389]:
rand_ind = int(input("Enter index: "))
print("The Data is : ")
panda = pd.Series([df["diagnosis_melanoma"][rand_ind],df["site_oral/genital"][rand_ind],df["site_upper extremity"][rand_ind],df["age_approx"][rand_ind]],["diagnosis_melanoma","site_oral/genity","site_upper extremity","age_approx"])

print(panda)
predicted_output = classifier_svm.predict([[df["diagnosis_melanoma"][rand_ind],df["site_oral/genital"][rand_ind],df["site_upper extremity"][rand_ind],df["age_approx"][rand_ind]]])
predicted_output = "Malignant" if predicted_output else "Benign"
actual_output = "Malignant" if df["benign_malignant"][rand_ind] else "Benign"
print("The Predicted Value is ",predicted_output)
print("The Actual Value is ",actual_output)
